In [43]:
!pip install folium
!pip install pillow


Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable


In [ ]:
import folium
from folium import Marker
import json
import requests

### 530 bus dataset 


In [14]:
app_id = "110703014-6dcbb76e-11e4-46d4"
app_key = "44ecf82b-641e-4a9d-859b-07f708481690"

In [30]:
auth_url="https://tdx.transportdata.tw/auth/realms/TDXConnect/protocol/openid-connect/token"
shape_url = "https://tdx.transportdata.tw/api/basic/v2/Bus/Shape/City/Taipei/530?%24top=30&%24format=JSON"
stop_url = "https://tdx.transportdata.tw/api/basic/v2/Bus/RealTimeByFrequency/City/Taipei/530?%24top=30&%24format=JSON"

In [29]:
class Auth():

    def __init__(self, app_id, app_key):
        self.app_id = app_id
        self.app_key = app_key

    def get_auth_header(self):
        content_type = 'application/x-www-form-urlencoded'
        grant_type = 'client_credentials'

        return{
            'content-type' : content_type,
            'grant_type' : grant_type,
            'client_id' : self.app_id,
            'client_secret' : self.app_key
        }

In [31]:
class data():

    def __init__(self, app_id, app_key, auth_response):
        self.app_id = app_id
        self.app_key = app_key
        self.auth_response = auth_response

    def get_data_header(self):
        auth_JSON = json.loads(self.auth_response.text)
        access_token = auth_JSON.get('access_token')

        return{
            'authorization': 'Bearer ' + access_token,
            'Accept-Encoding': 'gzip'
        }

In [32]:
a = Auth(app_id, app_key)
auth_response = requests.post(auth_url, a.get_auth_header())
d = data(app_id, app_key, auth_response)
data_response = requests.get(shape_url, headers=d.get_data_header())

In [33]:
with open("shape.json", "w", encoding="utf-8") as f:
        json.dump(data_response.text, f, ensure_ascii=False)

In [34]:
a = Auth(app_id, app_key)
auth_response = requests.post(auth_url, a.get_auth_header())
d = data(app_id, app_key, auth_response)
data_response = requests.get(stop_url, headers=d.get_data_header())

In [35]:
with open("stop.json", "w", encoding="utf-8") as f:
        json.dump(data_response.text, f, ensure_ascii=False)

### draw map

In [37]:
Taipei_map = folium.Map(location=[24.9812149808253, 121.585421349811], tiles='openstreetmap', zoom_start=15)
Taipei_map

In [38]:
with open("shape.json", "r", encoding="utf-8") as shape_file, open("stop.json", "r", encoding="utf-8") as stop_file:
    shape = json.load(shape_file)
    stop = json.load(stop_file)

In [39]:
if isinstance(shape, str):
    shape = json.loads(shape)
if isinstance(stop, str):
    stop = json.loads(stop)

In [40]:
# Add bus stop to map
for bus in stop:
    lat = bus["BusPosition"]["PositionLat"]
    lon = bus["BusPosition"]["PositionLon"]
    plate = bus["PlateNumb"]
    
    folium.Marker(
        location=[lat, lon],
        popup=f"車牌: {plate}",
        tooltip=plate,
        icon=folium.Icon(color="red", icon="info-sign")
    ).add_to(Taipei_map)
Taipei_map

In [41]:
from shapely.wkt import loads
line = loads(shape[0]["Geometry"])
coords = [(lat, lon) for lon, lat in line.coords]  
folium.PolyLine(coords, color="blue", weight=5, opacity=0.7).add_to(Taipei_map)
Taipei_map


In [44]:
from PIL import Image

# 讀取 PNG 圖片
image = Image.open("530_bus_trial.png")

# 轉換並存為 PDF
image.convert("RGB").save("530_bus_trial.pdf")